In [1]:
import pandas as pd
import numpy as np
import scipy as sp, scipy.stats
import warnings
warnings.filterwarnings("ignore")

In [3]:
# data_2010 = pd.read_csv('/Users/riyamhatre/Desktop/holidays_2010.csv')
# data_2011 = pd.read_csv('/Users/riyamhatre/Desktop/holidays_2011.csv')
# data = pd.concat([data_2010, data_2011]).drop(columns = {"Unnamed: 0"})

In [5]:
data = pd.read_csv('/Users/riyamhatre/Desktop/data.csv').drop(columns = {"Unnamed: 0"})

In [6]:
data.shape

(541909, 11)

In [7]:
data['Country']= data['Country'].replace("EIRE", "Ireland")

In [8]:
#Strings
data['Description'] = data['Description'].str.replace('.','').str.upper().str.strip()
data['InvoiceNo'] = data['InvoiceNo'].astype(str).str.upper()
data['Country'] = data['Country'].str.upper()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,CombinedDate,Holiday
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,UNITED KINGDOM,December,December 1,NaN
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,UNITED KINGDOM,December,December 1,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,UNITED KINGDOM,December,December 1,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,UNITED KINGDOM,December,December 1,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART,6,2010-12-01 08:26:00,3.39,17850.0,UNITED KINGDOM,December,December 1,NaN


In [9]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [10]:
# Remove Outliers Based on Z-score
data = data[(np.abs(sp.stats.zscore(data['UnitPrice']))<3) & (np.abs(sp.stats.zscore(data['Quantity']))<5)]

In [11]:
data.isnull().sum()

InvoiceNo            0
StockCode            0
Description       1447
Quantity             0
InvoiceDate          0
UnitPrice            0
CustomerID      134778
Country              0
Month                0
CombinedDate         0
Holiday         536034
dtype: int64

In [12]:
# Replace nan values in the Description column with appropriate values
data.drop(data[(data['Quantity']>0) & (data.InvoiceNo.str.contains('C') == True)].index, inplace = True)
data.drop(data[(data['Quantity']<0) & (data.InvoiceNo.str.contains('C') == False)].index, inplace = True)
data.drop(data[data['Description'].str.contains('?',regex=False) == True].index, inplace = True)
data.drop(data[data['UnitPrice'] == 0].index, inplace = True)

for index,value in data["StockCode"][data["Description"].isna()==True].items():
    if pd.notna(data["Description"][data["StockCode"] == value]).sum() != 0:
        data["Description"][index] = data["Description"][data["StockCode"] == value].mode()[0]
    else:
        data.drop(index = index, inplace = True)
        
data['Description'] = data['Description'].astype(str)

In [13]:
# Fixing Duplicate CustomerIDs
for i,v in data.groupby('CustomerID')['Country'].unique().items():
    if len(v)>1:
        data['Country'][data['CustomerID'] == i] = data['Country'][data['CustomerID'] == i].mode()[0]

# Adding More Features
data['TotalPrice'] = data['Quantity']*data['UnitPrice']
data['Day'] = data['CombinedDate'].str.split(' ').apply(lambda x: x[1])
data["InvoiceDate"] = pd.to_datetime(data['InvoiceDate'])
data["DayofWeek"] = data['InvoiceDate'].dt.day_name()

In [15]:
# create a column that checks returns 
data['Returned'] = data['InvoiceNo'].apply(lambda x: True if x[0] == "C" else False)

In [17]:
# Group countries by region
regions = {
    "Western Europe": ["UNITED KINGDOM", "FRANCE", "NETHERLANDS", "IRELAND", "BELGIUM", "SWITZERLAND", "PORTUGAL", "SPAIN"],
    "Northern Europe": ["NORWAY", "DENMARK", "SWEDEN", "FINLAND", "ICELAND"],
    "Eastern Europe": ["POLAND", "LITHUANIA", "CZECH REPUBLIC"],
    "Southern Europe": ["ITALY", "CYPRUS", "GREECE", "MALTA", "AUSTRIA"],
    "Middle East": ["ISRAEL", "LEBANON", "UNITED ARAB EMIRATES", "SAUDI ARABIA", "BAHRAIN"],
    "East Asia": ["JAPAN", "CHINA"],
    "Southeast Asia": ["SINGAPORE"],
    "Oceania": ["AUSTRALIA"],
    "North America": ["CANADA", "USA"],
    "South America": ["BRAZIL"],
    "Other": ["CHANNEL ISLANDS", "EUROPEAN COMMUNITY", "UNSPECIFIED", "RSA"]
}

# Create the mapping using a dictionary comprehension
region_map = {country: region for region, countries in regions.items() for country in countries}

# Map the regions to the Country column and create a new Region column
data["Region"] = data["Country"].map(region_map)

In [18]:
# Export Data
# from pathlib import Path
# filepath = Path('/Users/riyamhatre/Desktop/cleaned_data.csv')
# filepath.parent.mkdir(parents=True, exist_ok = True)
# data.to_csv(filepath)